In [1]:
import json

import random

random.seed(42)

In [2]:
#functions

def merge_json_files(json_files, output_file_path):
    merged_data = []

    for file in json_files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.extend(data)  # add the list from this file to the merged_data

    # Save the merged list to a new JSON file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, ensure_ascii=False, indent=2)

    return


In [10]:
all_training_data = []

In [11]:
#Open the blame data and make entry called label = 1

with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/json_files/sentences_in_five_templates.json", "r", encoding="utf-8") as f:
    blame_data = json.load(f)

for entry in blame_data:
    entry["label"] = 1
    all_training_data.append(entry)
    

In [12]:
with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/json_files/NO_BLAME_sentences_in_five_templates.json", "r", encoding="utf-8") as f:
    no_blame_data = json.load(f)

for entry in no_blame_data:
    entry["label"] = 0
    all_training_data.append(entry)

In [13]:
#save data

with open("/work/RuneEgeskovTrust#9638/Bachelor/training_data/training_data.json", "w", encoding="utf-8") as f:
        json.dump(all_training_data, f, ensure_ascii=False, indent=2)

In [14]:
len(all_training_data)

399018

In [15]:
# --- File paths ---
main_file = "/work/RuneEgeskovTrust#9638/Bachelor/training_data/training_data.json"          # The original dataset
exclude_file = "/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json"   # The dataset containing entries to delete
output_file = "/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data.json"


In [16]:
#delete entries also in validation data


# --- Load both files ---
with open(main_file, "r", encoding="utf-8") as f:
    main_data = json.load(f)

with open(exclude_file, "r", encoding="utf-8") as f:
    exclude_data = json.load(f)

# --- Create a set of (paragraph, sentence_nr) pairs to remove ---
exclude_pairs = {
    (str(entry.get("paragraph")), str(entry.get("sentence_nr")))
    for entry in exclude_data
}

# --- Filter out matching entries from the main dataset ---
filtered_data = [
    entry for entry in main_data
    if (str(entry.get("paragraph")), str(entry.get("sentence_nr"))) not in exclude_pairs
]

# --- Calculate and report how many were removed ---
deleted_count = len(main_data) - len(filtered_data)
print(f"Deleted {deleted_count} entries from the main dataset.")

# --- Save the cleaned data ---
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print(f"Cleaned dataset saved to {output_file}.")


Deleted 176 entries from the main dataset.
Cleaned dataset saved to /work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data.json.


In [ ]:
#make a subset of this dataset holding 1800 blame = 1 and 1800 blame = 0

def make_subset(n = 1800, base_data_file_path = '', subset_file_path = ''):
    with open(base_data_file_path, "r", encoding="utf-8") as f:
        full_training_data = json.load(f)


    # Separate entries by label
    label_0 = [entry for entry in full_training_data if entry["label"] == 0]
    label_1 = [entry for entry in full_training_data if entry["label"] == 1]

    # Randomly sample 2000 from each
    subset_0 = random.sample(label_0, n)
    subset_1 = random.sample(label_1, n)

    # Combine and shuffle
    subset = subset_0 + subset_1
    random.shuffle(subset)

    # Save subset
    with open(subset_file_path, "w") as f:
        json.dump(subset, f, indent=2)

    return

make_subset(n = 3600, 
            base_data_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data.json',
            subset_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/subset_cleaned_training_data.json')


In [ ]:
#Now make data for five and Four tempaltes

In [ ]:
def change_blame_by_template(base_file_path ='', mod_temp_path = '', output_path ='' ):
    # Load data
    with open(base_file_path, "r", encoding="utf-8") as f: 
        cl_tr_data = json.load(f)

    with open(mod_temp_path, "r", encoding="utf-8") as f:
        four_temps_blame = json.load(f)

    # Build a lookup set for fast matching
    four_temps_keys = {(item["paragraph"], item["sentence_nr"]) for item in four_temps_blame}

    # Track matches for validation
    found_count = 0

    for entry in cl_tr_data:
        key = (entry["paragraph"], entry["sentence_nr"])
        if key in four_temps_keys:
            if entry["label"] == 0:
                entry["label"] = 1
                found_count += 1
            else:
                raise ValueError(f"Entry {key} already has label=1 (should have been 0).")

    # Save updated data
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(cl_tr_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Updated {found_count} entries successfully.")

    return


change_blame_by_template(
    base_file_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data.json', 
    mod_temp_path = '/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/sentences_in_four_templates.json', 
    output_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_4_5_temps.json' )


In [15]:
#create subset of 3600

random.seed(52)

make_subset(n = 1800, 
            base_data_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_4_5_temps.json',
            subset_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/subset_4_5_cleaned_training_data.json')

In [ ]:
#Now for five, four and three templates

In [11]:
change_blame_by_template(
    base_file_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_4_5_temps.json', 
    mod_temp_path = '/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/sentences_in_three_templates.json', 
    output_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_3_4_5_temps.json' )

✅ Updated 768 entries successfully.


In [16]:
#make subset of this
random.seed(62)
make_subset(n = 1800, 
            base_data_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_3_4_5_temps.json',
            subset_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/subset_3_4_5_cleaned_training_data.json')

In [ ]:
#Now for five, four, three and two templates

In [17]:
change_blame_by_template(
    base_file_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_3_4_5_temps.json', 
    mod_temp_path = '/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/sentences_in_two_templates.json', 
    output_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_2_3_4_5_temps.json' )


#make subset of this
random.seed(72)
make_subset(n = 1800, 
            base_data_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_2_3_4_5_temps.json',
            subset_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/subset_2_3_4_5_cleaned_training_data.json')

✅ Updated 988 entries successfully.


In [ ]:
#Now for five, four, three, two and one templates

In [19]:
change_blame_by_template(
    base_file_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_2_3_4_5_temps.json', 
    mod_temp_path = '/work/MarkusLundsfrydJensen#1865/Bachelor_project/json_files/sentences_in_one_templates.json', 
    output_path ='/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_1_2_3_4_5_temps.json' )


#make subset of this
random.seed(82)
make_subset(n = 1800, 
            base_data_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_1_2_3_4_5_temps.json',
            subset_file_path = '/work/MarkusLundsfrydJensen#1865/Training_data/subset_1_2_3_4_5_cleaned_training_data.json')

✅ Updated 1998 entries successfully.


In [ ]:
#Hell yearh